# Parameters

# prepare package

In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
import importlib

In [ ]:
import Complex_Utility
importlib.reload(Complex_Utility)
from Complex_Utility import *

import monthly_bucket_report
importlib.reload(monthly_bucket_report)
from monthly_bucket_report import *

import config
importlib.reload(config)
from config import *

In [ ]:
os.chdir(f)
os.getcwd()

# Load

## Load data

In [ ]:
df_model = pd.read_pickle(score_file)
df = pd.read_pickle(data_f)

## Load model

In [ ]:
model = pickle.load(file = open(pkl_file,'rb'))
model_var = model.feature_name_
len(model_var)

## Data preparation

In [ ]:
'''离散变量处理'''
for col in list_dis:
    df[col] = df[col].astype('category')

## Prediction

In [ ]:
y_pred = model.predict_proba(df[model_var])[:,1]
df['y_pred'] = y_pred
y_pred_score = np.round(400-35/np.log(2)*np.log(y_pred/(1-y_pred)))
df['y_pred_score'] = y_pred_score

'''output full dataset'''
df.to_csv("result/%s_全量数据打分.csv" % part_nm, index = False)

# Subgroup

In [2]:
'''define subgroups'''
sub0 = (df['cd'] == 0)
sub1 = (df['cd'] == 1)
# df['subgroup'] = np.where(sub0, 'sub0', np.where(sub1, 'sub1', 'sub2'))
df['subgroup'] = np.select([sub0, sub1], ['sub0', 'sub1'], default= 'empty')

In [ ]:
'''choose subgroups'''
df_sample = df[df['subgroup'] == 'sub0']

# General report by Tag

In [ ]:
'''
tgt_col：y标签
y_pred: 评分或者概率
tag: 二级拆分标签。如：月份，城市等; 如果不需要，填空
group_column: 如果组标签是单个变量，就填此变量group_column；
                        如果组标签是分开的变量(如变量G1，G2, G3)，则填空
my_dict: eg {'G1':'y','G2':'y2','G3':'y3'} !!!: 请注意，如果是标签含重复的项，会被覆盖，使用下方my_pairs的方法替代dict格式。
    my_pairs = [('a', 'aa'), ('a', 'bb'), ('a', 'cc'), ('b', 'cc')]
'''

In [ ]:
'''
Functions Collections:
describe_report_tag(df, tag, tgt_col)
describe_report_tag2(df, tag1, tag2, tgt_col)
bads_table, total_prop_table, total_table, lift_table = bucket_distribution_tag(df, tag, y_pred, tgt_col, bench_mark, split_num, methods)
bads_table, total_prop_table, total_table, lift_table = bucket_distribution_tag(df, tag, y_pred, tgt_col, bench_mark, split_num, methods)
bucket_lift_tag(df, tag, y_pred, tgt_col, split_num, methods = 'quantile')
loop_report_subgroup(df, tag, y_pred, group_column, my_dict)
psi_ks_auc(df, tag, y_pred, tgt_col, bench_mark, split_num = 10)
'''

In [ ]:
a = describe_report_tag(df, yearmonth, y)

b = describe_report_tag2(df_sample, samp_col, yearmonth, y)

c = bucket_lift_tag(df, yearmonth, 'y_pred', y, split_num = 20, methods = 'quantile')

d = psi_ks_auc(df, yearmonth, 'y_pred', y, 'last_row', split_num = 10)

In [ ]:
%%capture --no-display
bads_table, total_prop_table, total_table, lift_table = bucket_distribution_tag(df, yearmonth, 'y_pred', y, '', 20, methods = 'quantile')

In [ ]:
my_pairs = [('credit', 'y_28'), ('credit', 'y_30'), ('credit', 'y_60'), ('mortgage', 'y_30')]
e = loop_report_subgroup(df, yearmonth, 'y_pred', 'sub_group', my_pairs)

In [ ]:
import File_utility
importlib.reload(File_utility)
from File_utility import *

In [ ]:
wb_sheet1 = wb.add_worksheet('%s_标签统计' % part_nm)
wb_sheet2 = wb.add_worksheet('%s_分月统计' % part_nm)
wb_sheet3 = wb.add_worksheet('%s_PSI_KS_AUC' % part_nm)
wb_sheet4 = wb.add_worksheet('%s_子客群报告' % part_nm)
wb_sheet5 = wb.add_worksheet('%s_分组分布' % part_nm)

In [ ]:
row_flag = 0
write_table(wb_sheet1, a, start_row=row_flag,start_col=0,ch_col=[0],
            str_col=[0],int_col=[1,2,3],decimal_col=[],pct_col=[4,5,6])

row_flag+=a.shape[0]+2
write_table(wb_sheet1, b, start_row=row_flag,start_col=0,ch_col=[0],
            str_col=[0,1],int_col=[2,3,4],decimal_col=[],pct_col=[5,6,7])

In [ ]:
write_table(wb_sheet2, c, start_row=0,start_col=0,ch_col=[0],
            str_col=[0],int_col=[1,2],decimal_col=[4,5,6],pct_col=[3])

write_table(wb_sheet3, d, start_row=0,start_col=0,ch_col=[0,1],
            str_col=[0,1],int_col=[],decimal_col=[2,3,4],pct_col=[])

write_table(wb_sheet4, e, start_row=0,start_col=0,ch_col=[0],
            str_col=[0,8],int_col=[1,2],decimal_col=[4,5,6,7],pct_col=[3])

In [ ]:
row_flag = 0
write_table(wb_sheet5, bads_table, start_row=row_flag,start_col=0,ch_col=[0],
            str_col=[0],int_col=[1,2,3,4,5,6,7,8,9,10,11,12],decimal_col=[],pct_col=[])

row_flag+=bads_table.shape[0]+2
write_table(wb_sheet5, total_prop_table, start_row=row_flag,start_col=0,ch_col=[0],
            str_col=[0],int_col=[],decimal_col=[],pct_col=[1,2,3,4,5,6,7,8,9,10,11,12])

row_flag+=total_prop_table.shape[0]+2
write_table(wb_sheet5, total_table, start_row=row_flag,start_col=0,ch_col=[0],
            str_col=[0],int_col=[1,2,3,4,5,6,7,8,9,10,11,12],decimal_col=[],pct_col=[])

row_flag+=total_table.shape[0]+2
write_table(wb_sheet5, lift_table, start_row=row_flag,start_col=0,ch_col=[0],
            str_col=[0],int_col=[],decimal_col=[1,2,3,4,5,6,7,8,9,10,11,12],pct_col=[])

In [ ]:
wb.close()
del wb